In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.0 MB/s 
     |████████████████████████████████| 120 kB 69.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.7 MB/s 


In [3]:
import pandas as pd
import os
from transformers import pipeline
from torch import nn
from transformers import BertModel
from transformers import BertTokenizer
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
os.chdir('./drive/MyDrive/Colab Notebooks/PyTorch/lesson9')

In [5]:
sentiment = pipeline("text-classification", model='sismetanin/rubert-ru-sentiment-rusentiment')


In [6]:
print(sentiment('Я купил молоко'))
print(sentiment('Это отличный магазин'))
print(sentiment('Всё ужастно, я больше ни когда туда не пойду'))
print(sentiment('Ну все,если поеду в Ачинск,я не увижусь с тем с кем давно хотела.черт:('))

[{'label': 'LABEL_1', 'score': 0.9653890132904053}]
[{'label': 'LABEL_2', 'score': 0.9854725003242493}]
[{'label': 'LABEL_0', 'score': 0.9562910199165344}]
[{'label': 'LABEL_0', 'score': 0.860710084438324}]


## Вариант работы из "коробки"

In [7]:
def pred_(txt):
  sent = sentiment(txt)[0]['label']
  return 0 if sent == 'LABEL_0' else 1

In [8]:
df = pd.read_csv("val.csv")

In [9]:
df['class'].value_counts()

1    11449
0    11234
Name: class, dtype: int64

In [10]:
df.head(3)

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0


In [11]:
df['prediction'] = df['text'].apply(lambda x: pred_(x))

In [12]:
accuracy = (df['class']==df['prediction']).astype(int).sum() / df['prediction'].count()

In [13]:
print(f'Accuracy у модели из коробки: {accuracy}')

Accuracy у модели из коробки: 0.8409381475113521


## Обучение сети

In [6]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [17]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-ru-sentiment-rusentiment')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=30,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    
    def __len__(self):
        return len(self._txts)
    

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [18]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [20]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.2):
        super().__init__()
        self.bert = BertModel.from_pretrained('sismetanin/rubert-ru-sentiment-rusentiment')
        layers = []
        layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(768, 512))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))        
        layers.append(nn.Linear(512, 2))
        layers.append(nn.Sigmoid())
        self.net = nn.Sequential(*layers)


    def forward(self, x, mask):
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        final_layer = self.net(pooled_output)
        return final_layer

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [30]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.net.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at sismetanin/rubert-ru-sentiment-rusentiment were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
print(model)
print("Parameters transfer learning:", sum([param.nelement() for param in model.net.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [32]:
from tqdm import tqdm

In [33]:
for epoch_num in range(5):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [15:08<00:00,  3.12it/s]


Epochs: 1 | Train Loss:  0.007         | Train Accuracy:  0.887         | Val Loss:  0.006         | Val Accuracy:  0.902


100%|██████████| 2836/2836 [15:06<00:00,  3.13it/s]


Epochs: 2 | Train Loss:  0.006         | Train Accuracy:  0.894         | Val Loss:  0.006         | Val Accuracy:  0.904


100%|██████████| 2836/2836 [15:04<00:00,  3.14it/s]


Epochs: 3 | Train Loss:  0.006         | Train Accuracy:  0.894         | Val Loss:  0.006         | Val Accuracy:  0.903


100%|██████████| 2836/2836 [15:05<00:00,  3.13it/s]


Epochs: 4 | Train Loss:  0.006         | Train Accuracy:  0.895         | Val Loss:  0.006         | Val Accuracy:  0.907


100%|██████████| 2836/2836 [15:00<00:00,  3.15it/s]


Epochs: 5 | Train Loss:  0.006         | Train Accuracy:  0.895         | Val Loss:  0.006         | Val Accuracy:  0.903
